In [1]:
import numpy as np
import pandas as pd
import math
from sklearn import metrics

from scipy.stats import entropy

import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import matplotlib.dates as mdates #to format dates on our plots
%matplotlib inline
import seaborn as sns
import wrangle

# This is to make sure matplotlib doesn't throw the following error:
# The next line fixes "TypeError: float() argument must be a string or a number, not 'Timestamp' matplotlib"
pd.plotting.register_matplotlib_converters()

print("Initalized")

Initalized


In [2]:
help(wrangle)

Help on module wrangle:

NAME
    wrangle

FUNCTIONS
    student_data()
        Files for this ask were provided off network and connect be retrived from the web. You must request access to them from the DBA
        
        * Pulls in the CodeUp anonymized student access data from a text file into a DF and renames the columns .
        * Merges the cohort data from the cohort.csv file with the student data
        * Sets the index to datetime
        * Creates a classtype, program, and location columns from program_id...
        and knowing which cohort belongs to which program ID
        * Creates seperate data frames for webdev and data science (ds) users
        * returns data, webdev, ds, and cohort dataframes (must grab all in this order)

FILE
    c:\users\anthony\documents\data_science\codeup\anomaly-project\wrangle.py




In [3]:
# pull in the dataframes from the wrangle function
data, web, ds, cohort = wrangle.student_data()
# print the heads of the data frames to ensure the wrangle function worked properly
data.head(1)

,date,time,page_viewed,user_id,cohort_id,ip,hour,day,month,name,start_date,end_date,program_id,classtype,program,location
datetime,,,,,,,,,,,,,,,,
2018-01-26 09:55:03,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,9,Friday,January,Hampton,2015-09-22,2016-02-06,1.0,Web-SanAntonio,Web,SanAntonio


In [4]:
web.head(1)

,date,time,page_viewed,user_id,cohort_id,ip,hour,day,month,name,start_date,end_date,program_id,classtype,program,location
datetime,,,,,,,,,,,,,,,,
2018-01-26 09:55:03,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,9,Friday,January,Hampton,2015-09-22,2016-02-06,1.0,Web-SanAntonio,Web,SanAntonio


In [5]:
ds.head(1)

,date,time,page_viewed,user_id,cohort_id,ip,hour,day,month,name,start_date,end_date,program_id,classtype,program,location
datetime,,,,,,,,,,,,,,,,
2019-08-20 09:39:58,2019-08-20,09:39:58,/,466,34.0,97.105.19.58,9,Tuesday,August,Bayes,2019-08-19,2020-01-30,3.0,DataScience-SanAntonio,DataScience,SanAntonio


In [6]:
cohort.head(1)

,cohort_id,name,start_date,end_date,program_id
0,1,Arches,2014-02-04,2014-04-22,1


# Monthly Data
To make the data processing easier we will resample the data by month and get the size (number of hits) for each cohort and page. We will use the ds_month and web_month in much of our analysis.

In [7]:
# resample the index by month and set reset the index to the 'datetime' column 
ds_month = ds.groupby(['name', 'page_viewed']).resample('M').size().reset_index().set_index('datetime')
web_month = web.groupby(['name', 'page_viewed']).resample('M').size().reset_index().set_index('datetime')
# Rename the colums to name (cohort name), page (page_viewed), and hits (.size() result)
ds_month.columns=['name', 'page', 'hits']
web_month.columns=['name', 'page', 'hits']

Print each newly created data frame to ensure we get the results we expect

In [8]:
web_month

,name,page,hits
datetime,,,
2019-03-31,Andromeda,/,132
2019-04-30,Andromeda,/,232
2019-05-31,Andromeda,/,188
2019-06-30,Andromeda,/,110
2019-07-31,Andromeda,/,108
...,...,...,...
2019-07-31,Zion,uploads/598dc43df39e2.jpg,1
2019-12-31,Zion,web-design/intro,1
2019-07-31,Zion,web-design/ui/visuals,1


In [9]:
ds_month

,name,page,hits
datetime,,,
2019-08-31,Bayes,/,255
2019-09-30,Bayes,/,304
2019-10-31,Bayes,/,326
2019-11-30,Bayes,/,228
2019-12-31,Bayes,/,211
...,...,...,...
2020-10-31,Darden,timeseries/working-with-time-series-data,75
2020-11-30,Darden,timeseries/working-with-time-series-data,8
2020-10-31,Darden,timeseries/working-with-time-series-data-with-...,2


# Consistently Viewed
> Which lesson appears to attract the most traffic consistently across cohorts (per program)?


To answer this question, I will take the ds and webdev programs and determine the total number of times each page was viewed by each cohort. After determining the number of times each cohort viewed each webpage I will get the total of views of the web page across the cohorts and then take the average. 

### Data Science(ds) Conistently Viewed

In [10]:
# group by cohort name and page and get the total number of hits per page per cohort
# Unstanck the dataframe to make the pages columns and reset column index to level one to access the page names directly
ds_sumhits = ds_month.groupby(['name', 'page']).sum().unstack().reset_index(col_level=1)
# Drop the top level column index as it's not needed
ds_sumhits.columns = ds_sumhits.columns.droplevel(0)
# reset the index
ds_sumhits = ds_sumhits.reset_index()
# transpose the results
ds_sumhits = ds_sumhits.transpose()
# drop the index and name rows as they are now extra
ds_sumhits = ds_sumhits.drop(labels=['index', 'name'])
# reset the index again and then set the index to page and drop the extra column created from this action
ds_sumhits = ds_sumhits.reset_index().set_index('page', drop=True)
# Grab the names of the cohorts from the ds_month dataframe as ds_columns
ds_columns=ds_month.name.unique()
# Assign the columns in the dataframe to the ds_columns
ds_sumhits.columns=ds_columns
# Fill all na values with 0
ds_sumhits = ds_sumhits.fillna(0)
# calculate the total number of hits across all cohorts
ds_sumhits['total_hits'] = ds_sumhits.sum(axis=1)
# Determine the average across all cohorts
ds_sumhits['total_mean'] = ds_sumhits.mean(axis=1)
# Print the newly created data frame where the average is above 300 views.
ds_sumhits[ds_sumhits['total_mean'] >= 300]

,Bayes,Curie,Darden,total_hits,total_mean
page,,,,,
/,1842.0,1523.0,2041.0,5406.0,2703.0
1-fundamentals/1.1-intro-to-data-science,615.0,461.0,460.0,1536.0,768.0
1-fundamentals/1.2-data-science-pipeline,221.0,146.0,271.0,638.0,319.0
1-fundamentals/AI-ML-DL-timeline.jpg,624.0,465.0,470.0,1559.0,779.5
1-fundamentals/DataToAction_v2.jpg,230.0,148.0,284.0,662.0,331.0
1-fundamentals/modern-data-scientist.jpg,626.0,467.0,470.0,1563.0,781.5
10-anomaly-detection/1-overview,383.0,345.0,14.0,742.0,371.0
10-anomaly-detection/AnomalyDetectionCartoon.jpeg,386.0,345.0,14.0,745.0,372.5
3-sql/1-mysql-overview,142.0,441.0,121.0,704.0,352.0


### WebDev Conistently Viewed

In [11]:
# group by cohort name and page and get the total number of hits per page per cohort
# Unstanck the dataframe to make the pages columns and reset column index to level one to access the page names directly
web_sumhits = web_month.groupby(['name', 'page']).sum().unstack().reset_index(col_level=1)
# Drop the top level column index as it's not needed
web_sumhits.columns = web_sumhits.columns.droplevel(0)
# reset the index
web_sumhits = web_sumhits.reset_index()
# transpose the results
web_sumhits = web_sumhits.transpose()
# drop the index and name rows as they are now extra
web_sumhits = web_sumhits.drop(labels=['index', 'name'])
# reset the index again and then set the index to page and drop the extra column created from this action
web_sumhits = web_sumhits.reset_index().set_index('page', drop=True)
# Grab the names of the cohorts from the web_month dataframe as web_columns
web_columns=web_month.name.unique()
# Assign the columns in the dataframe to the web_columns
web_sumhits.columns=web_columns
# Fill all na values with 0
web_sumhits = web_sumhits.fillna(0)
# calculate the total number of hits across all cohorts 
web_sumhits['total_hits'] = web_sumhits.sum(axis=1)
# calculate the average total_hits across the cohorts
web_sumhits['total_mean'] = web_sumhits.mean(axis=1)
# Print the newly created data frame where the average is above 300 views.
web_sumhits[web_sumhits['total_mean'] >= 300]

,Andromeda,Apex,Apollo,Arches,Badlands,Bash,Betelgeuse,Ceres,Deimos,Denali,...,Staff,Teddy,Ulysses,Voyageurs,Wrangell,Xanadu,Yosemite,Zion,total_hits,total_mean
page,,,,,,,,,,,,,,,,,,,,,
/,1156.0,1244.0,1.0,622.0,17.0,532.0,868.0,1620.0,1319.0,1.0,...,4633.0,1754.0,1618.0,2098.0,1112.0,916.0,962.0,1756.0,30668.0,1614.105263
appendix,161.0,426.0,0.0,133.0,0.0,80.0,180.0,408.0,287.0,0.0,...,989.0,488.0,174.0,393.0,209.0,320.0,142.0,289.0,6678.0,351.473684
html-css,372.0,708.0,0.0,215.0,2.0,214.0,425.0,753.0,609.0,0.0,...,977.0,225.0,555.0,528.0,405.0,587.0,362.0,649.0,10363.0,545.421053
java-i,336.0,528.0,0.0,132.0,1.0,157.0,422.0,510.0,434.0,0.0,...,624.0,501.0,375.0,639.0,401.0,483.0,245.0,593.0,8699.0,457.842105
java-ii,395.0,595.0,0.0,167.0,0.0,190.0,444.0,667.0,506.0,0.0,...,884.0,494.0,423.0,754.0,323.0,501.0,268.0,613.0,9956.0,524.000000
java-iii,430.0,616.0,0.0,153.0,0.0,72.0,444.0,674.0,517.0,0.0,...,908.0,712.0,411.0,770.0,335.0,495.0,340.0,742.0,10610.0,558.421053
javascript-i,509.0,610.0,0.0,294.0,1.0,319.0,686.0,977.0,696.0,0.0,...,1263.0,589.0,523.0,884.0,553.0,718.0,437.0,868.0,13902.0,731.684211
javascript-ii,290.0,431.0,0.0,204.0,0.0,175.0,374.0,519.0,392.0,0.0,...,744.0,468.0,308.0,584.0,278.0,493.0,332.0,637.0,8367.0,440.368421
jquery,335.0,437.0,0.0,133.0,0.0,233.0,540.0,632.0,466.0,0.0,...,615.0,498.0,363.0,583.0,374.0,543.0,278.0,559.0,8846.0,465.578947


# Cohorts Referring to Lessons More than Other Cohorts
> Is there a cohort that referred to a lesson significantly more that other cohorts seemed to gloss over? 

To answer this question, we will use the web_hits and ds_hits DFs we created above to calculate the percentage of total hits of each cohort produced.

In [12]:
# Create a copy of the web_hits data frame called used
used = pd.DataFrame.copy(web_sumhits)
# using the web_columns object we created above, use a for loop to conduct turn each column into a percentage...
# of the total hits of each web page
for col in web_columns:
    used[col] = used[col]/used['total_hits']
# Drop the total_hits and total_mean columns
used = used.drop(columns=['total_hits', 'total_mean'])
# Transpose the data frame to make the cohorts the rows and the pages the columns 
used = used.transpose()
# drop the 'Staff' row
used = used.drop(labels='Staff', axis=0)
# Print the results
used

page,.git,.gitignore,.well-known/assetlinks.json,/,00_,00_index,01_intro,02_listing_files,03_file_paths,04_navigating_the_filesystem,...,web-design/ui/typography,web-design/ui/visuals,web-design/ux,web-design/ux/layout,web-design/ux/layout/.json,web-design/ux/purpose,web-dev-day-two,working-with-time-series-data,wp-admin,wp-login
Andromeda,0.0,0.0,0.0,0.037694,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.034749,0.050459,0.0,0.026316,0.0,0.037915,0.0,0.0,0.0,0.0
Apex,0.0,0.0,0.0,0.040563,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.077220,0.087156,0.0,0.074561,0.0,0.104265,0.0,0.0,0.0,0.0
Apollo,0.0,0.0,0.0,0.000033,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
Arches,0.0,0.0,0.0,0.020282,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
Badlands,0.0,0.0,0.0,0.000554,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
Bash,0.0,0.0,0.0,0.017347,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.004587,0.0,0.004386,0.0,0.000000,1.0,0.0,0.0,0.0
Betelgeuse,0.0,0.0,0.0,0.028303,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.100386,0.137615,0.0,0.166667,0.0,0.151659,0.0,0.0,0.0,0.0
Ceres,0.0,0.0,0.0,0.052824,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.127413,0.155963,0.0,0.149123,1.0,0.132701,0.0,0.0,0.0,0.0
Deimos,0.0,0.0,0.0,0.043009,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.057915,0.073394,0.0,0.065789,0.0,0.052133,0.0,0.0,0.0,0.0
Denali,0.0,0.0,0.0,0.000033,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0


In [13]:
# grab the pages from the used data frame
pages = used.columns
# create an empty dataframe that uses the index from the used DF as the index only
web_top_views = pd.DataFrame(index=used.index)
# use the pages object to create a for loop that loops through each colummn and merges the web_top_views data frame with...
# the cohort-page pairs that have over 10% (.1) of the total views of that page across the program
# using a left join (to keep null values) and the indexes of each DF as the merge keys
for page in pages:
    web_top_views = web_top_views.merge(right=(pd.DataFrame(used[page][used[page]>=.1])), how='left', left_index=True, right_index=True)
# Concat the total_hits of each web page from the web_sumhits DF
web_top_views = pd.concat([web_top_views, web_sumhits[['total_hits']].transpose()])
# Print the results
web_top_views


page,.git,.gitignore,.well-known/assetlinks.json,/,00_,00_index,01_intro,02_listing_files,03_file_paths,04_navigating_the_filesystem,...,web-design/ui/typography,web-design/ui/visuals,web-design/ux,web-design/ux/layout,web-design/ux/layout/.json,web-design/ux/purpose,web-dev-day-two,working-with-time-series-data,wp-admin,wp-login
Andromeda,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Apex,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.104265,NaN,NaN,NaN,NaN
Apollo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Arches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Badlands,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bash,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
Betelgeuse,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.100386,0.137615,NaN,0.166667,NaN,0.151659,NaN,NaN,NaN,NaN
Ceres,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.127413,0.155963,NaN,0.149123,1.0,0.132701,NaN,NaN,NaN,NaN
Deimos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Denali,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## WebDev Cohort Lessons Views

In [14]:
# Tranpose the DF to make the cohorts and the total hits rows the columns and the pages into the rows
web_top_views = web_top_views.transpose()
# eliminate any page that had under 500 total page views as we were told to find cohorts with significant traffic
web_top_views = web_top_views[web_top_views['total_hits']>= 500]
# retranspose the the DF so we can work on the df from a cohort perspective 
# This is necessary to make a mask that uses page values rather cohort values
web_top_views = web_top_views.transpose()
# create a mask where the average number of null values is lower than the max number of null values for page values
mask = web_top_views.isnull().mean() < ((len(web_top_views.index)-1)/(len(web_top_views.index)))
# retranspose the DF to then apply the mask to the DF on page values
web_top_views = web_top_views.transpose()
web_top_views = web_top_views[mask]
# create a row_mask that eliminates any row that has all null values
row_mask = web_top_views.isnull().mean(axis=0) < 1
# Tranpose the dataframe AGAIN to apply the mask to cohorts
web_top_views = web_top_views.transpose()
web_top_views = web_top_views[row_mask]
# Print the results for significantly more web view traffic for pages from different cohorts
web_top_views


page,appendix/capstone-workbook/sw-project-planning,appendix/documentation,appendix/extra-challenges/basic-challenges,appendix/further-reading/spring/authorization,appendix/further-reading/spring/devtools-configuration,appendix/professional-development/t-block-resume,appendix/slides,examples/css/messages.html,examples/javascript/ajax-store.html,html-css/css-i,...,javascript-i/testing-user-functions,jquery/simple-simon,mkdocs/search_index.json,search/search_index.json,slides/arrays,spring/extra-features/form-validation,spring/extra-features/json-response,spring/fundamentals/form-model-binding,spring/fundamentals/integration-tests,toc
Andromeda,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.102908,0.105159,NaN,NaN,NaN
Apex,0.113798,NaN,NaN,NaN,NaN,0.182094,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.110912,NaN,NaN,NaN,NaN,0.175749,NaN
Bash,NaN,NaN,0.14032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Betelgeuse,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ceres,0.146515,NaN,NaN,NaN,NaN,NaN,NaN,0.104668,NaN,NaN,...,0.108328,NaN,NaN,0.112134,NaN,NaN,NaN,NaN,0.123978,NaN
Deimos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.117166,NaN
Europa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Fortuna,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.231660,NaN,NaN,NaN,0.122616,NaN
Ganymede,NaN,NaN,NaN,NaN,NaN,0.224583,0.118644,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Hyperion,NaN,NaN,NaN,NaN,NaN,0.209408,NaN,NaN,NaN,0.10095,...,0.101340,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
used = pd.DataFrame.copy(ds_sumhits)
for col in ds_columns:
    used[col] = used[col]/used['total_hits']
used = used.drop(columns=['total_hits', 'total_mean'])
used = used.transpose()
used

page,%20https://github.com/RaulCPena,",%20https://github.com/RaulCPena",/,1-fundamentals,1-fundamentals/1.1-intro-to-data-science,1-fundamentals/1.2-data-science-pipeline,1-fundamentals/1.3-pipeline-demo,1-fundamentals/2.1-excel-overview,1-fundamentals/2.1-spreadsheets-overview,1-fundamentals/2.2-excel-functions,...,timeseries/modeling-lesson2,timeseries/overview,timeseries/prep,timeseries/project,timeseries/sarimax,timeseries/svr,timeseries/working-with-time-series-data,timeseries/working-with-time-series-data-with-pandas,toc,working-with-time-series-data
Bayes,0.0,0.0,0.340733,0.0,0.400391,0.346395,0.433333,0.459893,0.043011,0.711628,...,0.063830,0.012987,0.048780,0.000000,0.142857,0.125,0.023256,0.25,1.0,0.0
Curie,1.0,1.0,0.281724,0.0,0.300130,0.228840,0.200000,0.245989,0.784946,0.125581,...,0.234043,0.103896,0.195122,0.014286,0.285714,0.125,0.011628,0.25,0.0,0.0
Darden,0.0,0.0,0.377543,1.0,0.299479,0.424765,0.366667,0.294118,0.172043,0.162791,...,0.702128,0.883117,0.756098,0.985714,0.571429,0.750,0.965116,0.50,0.0,1.0


In [18]:
pages = used.columns
ds_top_views = pd.DataFrame(index=used.index)
for page in pages:
    ds_top_views = ds_top_views.merge(right=(pd.DataFrame(used[page][used[page]>=.1])), how='left', left_index=True, right_index=True)
ds_top_views = pd.concat([ds_top_views, ds_sumhits[['total_hits']].transpose()])
ds_top_views


page,%20https://github.com/RaulCPena,",%20https://github.com/RaulCPena",/,1-fundamentals,1-fundamentals/1.1-intro-to-data-science,1-fundamentals/1.2-data-science-pipeline,1-fundamentals/1.3-pipeline-demo,1-fundamentals/2.1-excel-overview,1-fundamentals/2.1-spreadsheets-overview,1-fundamentals/2.2-excel-functions,...,timeseries/modeling-lesson2,timeseries/overview,timeseries/prep,timeseries/project,timeseries/sarimax,timeseries/svr,timeseries/working-with-time-series-data,timeseries/working-with-time-series-data-with-pandas,toc,working-with-time-series-data
Bayes,NaN,NaN,0.340733,NaN,0.400391,0.346395,0.433333,0.459893,NaN,0.711628,...,NaN,NaN,NaN,NaN,0.142857,0.125,NaN,0.25,1.0,NaN
Curie,1.0,1.0,0.281724,NaN,0.300130,0.228840,0.200000,0.245989,0.784946,0.125581,...,0.234043,0.103896,0.195122,NaN,0.285714,0.125,NaN,0.25,NaN,NaN
Darden,NaN,NaN,0.377543,1.0,0.299479,0.424765,0.366667,0.294118,0.172043,0.162791,...,0.702128,0.883117,0.756098,0.985714,0.571429,0.750,0.965116,0.50,NaN,1.0
total_hits,1.0,1.0,5406.000000,10.0,1536.000000,638.000000,300.000000,187.000000,93.000000,215.000000,...,47.000000,77.000000,123.000000,70.000000,7.000000,8.000,86.000000,8.00,9.0,14.0


## DS Cohort Lessons Views
Repeat the same process above, but for data science cohorts

In [20]:
# Tranpose the DF to make the cohorts and the total hits rows the columns and the pages into the rows
ds_top_views = ds_top_views.transpose()
# eliminate any page that had under 500 total page views as we were told to find cohorts with significant traffic
ds_top_views = ds_top_views[ds_top_views['total_hits']>= 500]
# retranspose the the DF so we can work on the df from a cohort perspective 
# This is necessary to make a mask that uses page values rather cohort values
ds_top_views = ds_top_views.transpose()
# create a mask where the average number of null values is lower than the max number of null values for page values
mask = ds_top_views.isnull().mean() < ((len(ds_top_views.index)-1)/(len(ds_top_views.index)))
# retranspose the DF to then apply the mask to the DF on page values
ds_top_views = ds_top_views.transpose()
ds_top_views = ds_top_views[mask]
# create a row_mask that eliminates any row that has all null values
row_mask = ds_top_views.isnull().mean(axis=0) < 1
# Tranpose the dataframe AGAIN to apply the mask to cohorts
ds_top_views = ds_top_views.transpose()
ds_top_views = ds_top_views[row_mask]
# Print the results for significantly more web view traffic for pages from different cohorts
ds_top_views


page,/,1-fundamentals/1.1-intro-to-data-science,1-fundamentals/1.2-data-science-pipeline,1-fundamentals/AI-ML-DL-timeline.jpg,1-fundamentals/DataToAction_v2.jpg,1-fundamentals/modern-data-scientist.jpg,10-anomaly-detection/1-overview,10-anomaly-detection/AnomalyDetectionCartoon.jpeg,3-sql/1-mysql-overview,5-stats/3-probability-distributions,5-stats/4.2-compare-means,6-regression/1-overview,appendix/cli-git-overview,classification/overview,classification/scale_features_or_not.svg,search/search_index.json,sql/mysql-overview
Bayes,0.340733,0.400391,0.346395,0.400257,0.347432,0.400512,0.516173,0.518121,0.201705,0.609524,0.568592,0.463936,0.537133,NaN,NaN,0.413974,NaN
Curie,0.281724,0.300130,0.228840,0.298268,0.223565,0.298784,0.464960,0.463087,0.626420,0.331429,0.355596,0.529831,0.328152,0.105814,NaN,0.360631,0.160976
Darden,0.377543,0.299479,0.424765,0.301475,0.429003,0.300704,NaN,NaN,0.171875,NaN,NaN,NaN,0.134715,0.882558,0.919003,0.225394,0.834146
total_hits,5406.000000,1536.000000,638.000000,1559.000000,662.000000,1563.000000,742.000000,745.000000,704.000000,525.000000,554.000000,1123.000000,579.000000,860.000000,642.000000,1331.000000,615.000000


# Lest Accessed Lessons (web-pages)
> Which lessons are least accessed?

To answer this question we get the total number of page views (hits) by each program and pull out the pages who's views were below the second quartile 

In [21]:
# create a data frame from the web program that pulls the total number of page views (hits)
web_lesson = pd.DataFrame(web.groupby(['page_viewed']).size())
# rename the column to hits to be consistent
web_lesson.columns=['hits']
# describe the data frame to get the quartiles
web_lesson.describe()

,hits
count,1939.000000
mean,312.842702
std,1343.568915
min,1.000000
25%,2.000000
50%,8.000000
75%,35.000000
max,30668.000000


## WebDev Lest Accessed Lessons

In [22]:
web_lesson[web_lesson['hits']<8]

,hits
page_viewed,
.git,1
.gitignore,1
.well-known/assetlinks.json,2
00_,1
06_moving_files,7
...,...
web-design/ux/layout/.json,1
web-dev-day-two,2
working-with-time-series-data,1


In [23]:
# create a data frame from the web program that pulls the total number of page views (hits)
ds_lesson = pd.DataFrame(ds.groupby(['page_viewed']).size())
# rename the column to hits to be consistent
ds_lesson.columns=['hits']
# describe the data frame to get the quartiles
ds_lesson.describe()

,hits
count,596.000000
mean,114.120805
std,282.876447
min,1.000000
25%,3.000000
50%,20.000000
75%,160.000000
max,5406.000000


## DS Lest Accessed Lessons

In [24]:
ds_lesson[ds_lesson['hits']<20]

,hits
page_viewed,
%20https://github.com/RaulCPena,1
",%20https://github.com/RaulCPena",1
1-fundamentals,10
1-fundamentals/2.3-visualization,4
1-fundamentals/2.3-visualization.md,5
...,...
timeseries/sarimax,7
timeseries/svr,8
timeseries/working-with-time-series-data-with-pandas,8


# Students Hardly Viewing Lessons
> Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students

To answer this question we will create a WebDev and DS DFs that holds all the data of each user (students) and cohort when the cohort was in session. From there we will determine what users bare accessed the lessons.  